# Efficient Yelp API Calls (Core)
- Martisha Owens
- 11/16/2022

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI

from tqdm.notebook import tqdm_notebook
from tqdm.notebook import tqdm_notebook
import time

## Credentials and Accessing the API

In [18]:
with open('/Users/martishaowens/.secret/yelp_api.json') as f:
    login = json.load(f)
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

## Set API call parameters 

In [19]:
# set our API call parameters 
LOCATION = 'Paris, France'
TERM = 'Sushi'

## Specifying JSON_FILE filename

In [20]:
JSON_FILE = "Data/results_in_progress_Paris_Sushi.json"
JSON_FILE

'Data/results_in_progress_Paris_Sushi.json'

## For Loop to call each page

In [21]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == True:        
        if delete_if_exists==True:          
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
                        
    else:        
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        folder = os.path.dirname(JSON_FILE)        
        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [22]:
create_json_file(JSON_FILE, delete_if_exists=True)
        
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

total_results = results['total']
results_per_page = len(results['businesses'])
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_Paris_Sushi.json not found. Saving empty list to new file.
- 0 previous results found.


32

In [23]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
        
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
        
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/32 [00:00<?, ?it/s]

## Convert .json to dataframe

In [24]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,PHsKPp496sFZm9K9Xz_TlA,sanukiya-paris,Sanukiya,https://s3-media2.fl.yelpcdn.com/bphoto/va06K7...,False,https://www.yelp.com/biz/sanukiya-paris?adjust...,364,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.5,"{'latitude': 48.8647, 'longitude': 2.3338199}",[],€€,"{'address1': '9 rue d'Argenteuil', 'address2':...",+33142605261,+33 1 42 60 52 61,687.128906
1,qNdmSOThb2nu0c99NcyEtg,takara-paris-2,Takara,https://s3-media2.fl.yelpcdn.com/bphoto/EBK8N7...,False,https://www.yelp.com/biz/takara-paris-2?adjust...,59,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.5,"{'latitude': 48.864977, 'longitude': 2.335903}",[],€€€€,"{'address1': '14 rue Molière', 'address2': Non...",+33142960838,+33 1 42 96 08 38,566.853169
2,m7kDVQc-1_yax-KFA7pz0g,sushi-yaki-paris-4,Sushi Yaki,https://s3-media3.fl.yelpcdn.com/bphoto/jVovGw...,False,https://www.yelp.com/biz/sushi-yaki-paris-4?ad...,53,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 48.84845, 'longitude': 2.34066}",[],€€,"{'address1': '50 rue Monsieur le Prince', 'add...",+33146330189,+33 1 46 33 01 89,1506.025859
3,FXCx3Tuj8NJfCKPHJzyakg,okuda-paris,Okuda,https://s3-media4.fl.yelpcdn.com/bphoto/pz8j-s...,False,https://www.yelp.com/biz/okuda-paris?adjust_cr...,6,"[{'alias': 'japanese', 'title': 'Japanese'}]",5.0,"{'latitude': 48.8670149, 'longitude': 2.3028129}",[],NaN,"{'address1': '7 rue de la Trémoille', 'address...",+33140701919,+33 1 40 70 19 19,2931.712740
4,4LFbvZA5MINyPNoA8uLHzw,le-bar-à-sushi-izumi-paris-3,Le Bar à Sushi Izumi,https://s3-media1.fl.yelpcdn.com/bphoto/1TyTVq...,False,https://www.yelp.com/biz/le-bar-%C3%A0-sushi-i...,18,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 48.8814583, 'longitude': 2.3183}",[],€€€,"{'address1': '55 bd des Batignolles', 'address...",+33145224355,+33 1 45 22 43 55,2781.449109


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
619,UYBLYvRSpjKmu4Rj90SmVw,sen-sushi-sucy-en-brie,Sen Sushi,https://s3-media1.fl.yelpcdn.com/bphoto/vQqoEe...,False,https://www.yelp.com/biz/sen-sushi-sucy-en-bri...,1,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.0,"{'latitude': 48.7776339283437, 'longitude': 2....",[],NaN,"{'address1': '78 rue du général Leclerc', 'add...",+33156732920,+33 1 56 73 29 20,15556.827709
620,BVZXB7dAmYeZxsRpKCl2xw,sushi-samouraï-6-bourg-la-reine,Sushi Samouraï 6,,False,https://www.yelp.com/biz/sushi-samoura%C3%AF-6...,3,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.0,"{'latitude': 48.787777, 'longitude': 2.318381}",[],NaN,"{'address1': '6 ave du Général Leclerc', 'addr...",+33146653237,+33 1 46 65 32 37,8430.110168
621,WzhE-627ZTgf1jYgH0HJ2g,osaka-levallois-perret,Osaka,,False,https://www.yelp.com/biz/osaka-levallois-perre...,6,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",3.0,"{'latitude': 48.890742, 'longitude': 2.286481}",[],€€,"{'address1': '52 rue Anatole France', 'address...",+33147586712,+33 1 47 58 67 12,5178.468952
622,7v9SyKyf7mfkTSPM8O1wyA,oz-restaurant-neuilly-sur-seine,Oz Restaurant,,False,https://www.yelp.com/biz/oz-restaurant-neuilly...,1,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",2.0,"{'latitude': 48.8873567, 'longitude': 2.2602371}",[],NaN,"{'address1': '19 rue Beffroy', 'address2': '',...",+33147475310,+33 1 47 47 53 10,6622.771962
623,YRNj7L4fyTKpFF1HYP-bBw,mendels-vincennes,Mendel's,,False,https://www.yelp.com/biz/mendels-vincennes?adj...,2,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.0,"{'latitude': 48.8487, 'longitude': 2.4299}",[],NaN,"{'address1': '175 Rue de Fontenay', 'address2'...",+33143981717,+33 1 43 98 17 17,6587.225386


## Check for duplicates

In [25]:
final_df.duplicated(subset='id').sum()

0

## Save the final DataFrame to a .csv 

In [26]:
final_df.to_csv('Data/final_results_Paris_Sushi.csv.gz', compression='gzip',index=False)